In [1]:
import importlib
import brikasutils as bu
importlib.reload(bu)
import shared_utils as utils
from shared_utils import systemMsg, userMsg, assistantMsg
importlib.reload(utils)
import survey
importlib.reload(survey)
import persona
importlib.reload(persona)
import ollama
import numpy as np
from numpy.linalg import norm
import pandas as pd
import os
import re
import time
import json
from typing import List
from openai import OpenAI

In [3]:
texts_with_elias = [
    "selected-data/elias/message_1.json",
]
texts_with_petyo = [
    "selected-data/petyo/message_1.json",
    "selected-data/petyo/message_2.json",
    "selected-data/petyo/message_3.json",
    "selected-data/petyo/message_4.json",
    "selected-data/petyo/message_5.json",
]
texts_with_others_dict = {
    "anna": ["selected-data/others/anna.json"],
    "patryk": ["selected-data/others/patryk.json"],
    "andreas": ["selected-data/others/andreas.json"],
    "victoria": ["selected-data/others/victoria.json", "selected-data/others/victoria2.json"],
    "joanna": ["selected-data/others/joanna.json"],
    "antoni": ["selected-data/others/antoni.json"],
    "arijan": ["selected-data/others/arijan.json"],
    "denis": ["selected-data/others/denis.json"],
    "alexandra": ["selected-data/others/alexandra.json"],
    "FED": ["selected-data/others/FED.json"],
    "filip": ["selected-data/others/filip.json"],
    "kuba": ["selected-data/others/kuba.json"],
    "laura": ["selected-data/others/laura.json"],
    "liisa": ["selected-data/others/liisa.json"],
    "luiza": ["selected-data/others/luiza.json"],
    "marcus": ["selected-data/others/marcus.json"],
}


et = persona.PersonaEncoder()
et.parse_fb_messages(texts_with_elias, "elias")
et.parse_fb_messages(texts_with_petyo, "petyo")
for name, texts in texts_with_others_dict.items():
    et.parse_fb_messages(texts, name)

# et.parse_rosebud_entries("selected-data/rosebud.md", "rosebud")
et.filter_chats_empty()
et.filter_chats_regex(utils.BLACKLIST_CHAT_REGEX_FILTERS)


for nameid, chat in et.chats.items():
    for msg in chat:  
        msg.sender = "Persona" if msg.sender == "Airidas Brikas" else "Friend"

# Start all chats from 2/3rds
# for nameid, chat in et.chats.items():
#     et.chats[nameid] = chat[int(len(chat)/3 * 2):]

# et.select_chat_limited_by_tokens("elias", 6000)
# et.select_chat_limited_by_tokens("petyo", 6000)
et.select_chat_full("elias")
et.select_chat_full("petyo")

for name in texts_with_others_dict.keys():
    et.select_chat_full(name)

# et.select_nonChat_module_full("rosebud")

token_counts = et.count_all_selected_chat_tokens() # token_counts used later for statistics
print(f"Combined tokens: {sum(token_counts.values())}")

Read 1946 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2021-09-13 to 2024-03-06
Messages saved to self.chats['elias']
Read 40036 messages from 5 files. Failed to read 0 messages.
Messages ranged from 2020-08-17 to 2024-03-04
Messages saved to self.chats['petyo']
Read 7953 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2023-05-12 to 2024-03-04
Messages saved to self.chats['anna']
Read 5734 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2020-09-18 to 2024-03-02
Messages saved to self.chats['patryk']
Read 372 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2022-03-19 to 2024-02-24
Messages saved to self.chats['andreas']
Read 3399 messages from 2 files. Failed to read 0 messages.
Messages ranged from 2021-08-23 to 2024-03-02
Messages saved to self.chats['victoria']
Read 2951 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2022-11-13 to 2024-02-19
Messages saved to self.ch

# Final Boss

In [20]:
CHUNK_SIZES = [75]
chunk_size = CHUNK_SIZES[0]
EMBED_MODEL = "nomic-embed-text"
OVERLAP_SIZES = [3]
overlap_size = OVERLAP_SIZES[0]

chunks = []
chunk_token_counts = []
for chat in et.selectedChats.values():
    messages = list(chat)  # Convert chat iterator to list for easier slicing
    num_messages = len(messages)
    for i in range(0, num_messages - chunk_size + 1, chunk_size - overlap_size):
        chunk = messages[i:i + chunk_size]  # Extract chunk of messages
        chunk_text = "\\n".join(str(msg) for msg in chunk)  # Concatenate msgs into a single string
        chunks.append(chunk_text)  # Append chunk to list of chunks
        chunk_token_counts.append(utils.count_tokens(chunk_text))  # Append token count of the chunk
avg_chunk_token_count = sum(chunk_token_counts) / len(chunk_token_counts)
embeddings = []
progress, chunks_len = 0, len(chunks)
for chunk_text in chunks:
    progress += 1
    print(f"\rChunk {progress}/{chunks_len}", end="")
    embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
    embeddings.append(embedding)

Chunk 929/929

In [31]:
EMBEDDING_NAMEID = "airidas_finalboss_1"
AUTO_INFO = {
    "model": EMBED_MODEL,
    "CHUNK_SIZE": chunk_size,
    "OVERLAP_SIZE": overlap_size,
    "chunks_count": len(chunks),
    "modules_chat": token_counts,
}

bu.quickJSON(AUTO_INFO, f"embeddings/{EMBEDDING_NAMEID}_info.json")
bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"embeddings/{EMBEDDING_NAMEID}_embeddings.json")

In [30]:
PROMPT_METHOD ="IMPERSONATE"
SUBJECT = "airidas"
RETRIAVAL_METHODS = ["dynamic"]#, "hybrid"]
NUM_RUNS = 3
# MODEL = "llama3-70b"
# MODEL = "llama3-8b"
MODEL = "mixtral-8x22b"
# max_tokens = [0, 4000, 7500]
max_tokens = [0, 4000, 31500]

survs = [survey.KanoSurvey(), survey.PersonalitySurvey()]
for surv in survs:
    if isinstance(surv, survey.KanoSurvey):
        DYNAMIC_RETRIEVAL_PROMPTS = list(surv.questions)
        PROMPT_COUNT = 40
        SURVEY_TYPE = "KanoSurvey",
        WHICH_SURVEY = "kano"
        RETRIEVAL_PROMPT = "video game features"
        SURVEY = "video game preferences"
        METHOD = "a Kano survey"
    elif isinstance(surv, survey.PersonalitySurvey):
        DYNAMIC_RETRIEVAL_PROMPTS = list(surv.questions)
        PROMPT_COUNT = 50
        SURVEY_TYPE = "PersonalitySurvey",
        WHICH_SURVEY = "pers"
        RETRIEVAL_PROMPT = "openess conciousness extrovert aggreableness neuroticism"
        SURVEY = "personality traits"
        METHOD = "an OCEAN test"

    for max_token in max_tokens:

        for RETRIAVAL_METHOD in RETRIAVAL_METHODS:
            if RETRIAVAL_METHOD == "dynamic":
                dynamic_retrieval_prompts = list(surv.questions)
                dynamic_chunks_most_similar: List[List[str]] = [] 
                progress = 0
                lenn = len(dynamic_retrieval_prompts)
                for prompt in dynamic_retrieval_prompts:
                    progress += 1
                    print(f"\rPrompt {progress}/{lenn}", end="")
                    prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=prompt)["embedding"]

                    ## Chunking
                    max_chunks_count = int((max_token / avg_chunk_token_count))
                    chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)
                    chunks_most_similar = [chunks[embedding[1]] for embedding in chunks_most_similar_embeddings]

                    if max_token == 0:
                        chunks_most_similar = chunks_most_similar[:1]
                    else:
                        # Limit chunks by man token count
                        cur_tc = 0 # current token count
                        selected_chunks = []
                        for chunk in chunks_most_similar:
                            tk_in_chunk = utils.count_tokens(chunk)
                            if cur_tc + tk_in_chunk >= max_token:
                                break
                            cur_tc += tk_in_chunk
                            selected_chunks.append(chunk)
                        chunks_most_similar = selected_chunks
                    ##

                    tokens_in_chunks = sum(utils.count_tokens(chunk) for chunk in chunks_most_similar)
                    dynamic_chunks_most_similar.append(chunks_most_similar)
                print(end="\n")
                tokens_in_chunks = 0
                for chunks_most_similar in dynamic_chunks_most_similar:
                    for chunk in chunks_most_similar:
                        tokens_in_chunks += utils.count_tokens(chunk)
                del chunks_most_similar_embeddings  # free memory
                print(f"Tokens in average chunk group: {tokens_in_chunks/len(dynamic_chunks_most_similar)}")
                final_prompts = []
                prompt_template = """
for question, chunks_most_similar in zip(surv.questions, dynamic_chunks_most_similar):
    p = [
        systemMsg("\\n".join([
            f"You are an expert actor, specializing in impersonation of non-famous people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n#Context \\n##Chat conversations between the subject and their friends:\\n",
            "\\n\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
        ])),     
        assistantMsg("Understood. I will answer from the point of view of the persona, based on what I could the deduct from the text provided."),
        userMsg("\\n".join([
            f"Persona is questioned about their {SURVEY} in {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n",
            question,
            "\\nThe persona chooses:"
        ]))]
    final_prompts.append(p)
                """
            else: print("not dynamic")
            exec(prompt_template)
            SIM_ID = f"{SUBJECT}-{WHICH_SURVEY}-{str(max_token)}_{MODEL}_V7"
            bu.quickJSON(final_prompts, f"temp/{SIM_ID}_prompts.json")
            for num_run in range(NUM_RUNS):
                instructions = {
                    "prompt_file": f"batch/prompts/{SIM_ID}_prompts.json",
                    "survey_type": f"{SURVEY_TYPE[0]}",
                    "isLocal": True,
                    "LIMIT": None
                }
                settings = {
                    "model": MODEL,
                    "timeout": 300}
                AUTO_INFO = {
                    "CHUNK_SIZE": chunk_size,
                    "OVERLAP_SIZE": overlap_size,
                    "CTX_limit": max_token,
                    "chunk_count": len(chunks_most_similar),
                    "EMBED_MODEL": EMBED_MODEL,
                    "prompt method": PROMPT_METHOD,
                    "retrieval method": RETRIAVAL_METHOD,
                    "RETRIEVAL_PROMPT": RETRIEVAL_PROMPT,
                    "prompt_count": PROMPT_COUNT,
                    "survey": WHICH_SURVEY,
                    "SUBJECT": SUBJECT,
                    "prompt_template": prompt_template,
                    "CHUNKS_COUNT_IN_CTX": max_chunks_count,
                    **utils.describe_prompts(final_prompts)
                }
                bu.quickJSON({"instructions": instructions, "settings": settings, "info": AUTO_INFO}, f"temp/runs/{SIM_ID}_{num_run}.json")

Using default Kano Survey CSV file: surveys/survey_kano-model.csv
Using default Personality Survey CSV file: surveys/survey_personality-test.csv
Prompt 40/40
Tokens in average chunk group: 1189.575
Prompt 40/40
Tokens in average chunk group: 3450.625
Prompt 40/40
Tokens in average chunk group: 30952.125
Prompt 50/50
Tokens in average chunk group: 1162.62
Prompt 50/50
Tokens in average chunk group: 3272.88
Prompt 50/50
Tokens in average chunk group: 30877.2


In [6]:
chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)

In [18]:
max_tokens = 8000
cur_tc = 0 # current token count
selected_chunks = []
for chunk in chunks_most_similar:
    tk_in_chunk = utils.count_tokens(chunk)
    if cur_tc + tk_in_chunk >= max_tokens:
        break
    cur_tc += tk_in_chunk
    selected_chunks.append(chunk)
print(f"Tokens in chunks: {cur_tc}")
len(selected_chunks)

Tokens in chunks: 6947


7

### V5/6 - 32k models w/ timestamp

In [ ]:
CHUNK_SIZES = [250]
MODELS = ["mixtral:8x22b-instruct-v0.1-q2_K"]
# OVERLAP_SIZES = [5]
overlap_size = 5
RETRIAVAL_METHODS = ["dynamic", "hybrid"]
max_tokens = 10000
for chunk_size in CHUNK_SIZES:
    for MODEL in MODELS:
        chunks = []
        chunk_token_counts = []
        for chat in et.selectedChats.values():
            messages = list(chat)  # Convert chat iterator to list for easier slicing
            num_messages = len(messages)
            for i in range(0, num_messages - chunk_size + 1, chunk_size - overlap_size):
                chunk = messages[i:i + chunk_size]  # Extract chunk of messages
                chunk_text = "\\n".join(str(msg) for msg in chunk)  # Concatenate msgs into a single string
                chunks.append(chunk_text)  # Append chunk to list of chunks
                chunk_token_counts.append(utils.count_tokens(chunk_text))  # Append token count of the chunk
        avg_chunk_token_count = sum(chunk_token_counts) / len(chunk_token_counts)
        embeddings = []
        progress, chunks_len = 0, len(chunks)
        for chunk_text in chunks:
            progress += 1
            print(f"\rChunk {progress}/{chunks_len}", end="")
            embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
            embeddings.append(embedding)
        for RETRIAVAL_METHOD in RETRIAVAL_METHODS:
            if RETRIAVAL_METHOD == "dynamic":
                dynamic_retrieval_prompts = list(surv.questions)
                dynamic_chunks_most_similar: List[List[str]] = [] 
                progress = 0
                lenn = len(dynamic_retrieval_prompts)
                for prompt in dynamic_retrieval_prompts:
                    progress += 1
                    print(f"\rPrompt {progress}/{lenn}", end="")
                    prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=prompt)["embedding"]
                    max_chunks_count = int((max_tokens / avg_chunk_token_count)-1)
                    chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)[:max_chunks_count]
                    chunks_most_similar = [chunks[embedding[1]] for embedding in chunks_most_similar_embeddings]
                    tokens_in_chunks = sum(utils.count_tokens(chunk) for chunk in chunks_most_similar)
                    dynamic_chunks_most_similar.append(chunks_most_similar)
                print(end="\n")
                tokens_in_chunks = 0
                for chunks_most_similar in dynamic_chunks_most_similar:
                    for chunk in chunks_most_similar:
                        tokens_in_chunks += utils.count_tokens(chunk)
                del chunks_most_similar_embeddings  # free memory
                print(f"Tokens in average chunk group: {tokens_in_chunks/len(dynamic_chunks_most_similar)}")
                final_prompts = []
                prompt_template = """
for question, chunks_most_similar in zip(surv.questions, dynamic_chunks_most_similar):
    p = [
        systemMsg("\\n".join([
            f"You are an expert actor, specializing in impersonation of non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n#Context \\n##Chat conversations between the subject and their friends:\\n**From most to least related**\\n",
            "\\n\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
        ])),      
        assistantMsg("Understood. I will answer from the point of view of the persona, based on what I could the deduct from the text provided."),
        userMsg("\\n".join([
            f"Persona is questioned about their {SURVEY} in an {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n",
            question,
            "\\nThe persona chooses:"
        ]))]
    final_prompts.append(p)
                """
            elif RETRIAVAL_METHOD == "hybrid":
                prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=RETRIEVAL_PROMPT)["embedding"]
                max_chunks_count = int((max_tokens / avg_chunk_token_count))
                chunks_most_similar_embeddings_static = utils.find_most_similar(prompt_embedding, embeddings)[:max_chunks_count // 2]
                chunks_most_similar_static = [chunks[embedding[1]] for embedding in chunks_most_similar_embeddings_static]

                dynamic_retrieval_prompts = list(surv.questions)
                dynamic_chunks_most_similar: List[List[str]] = []
                progress = 0
                lenn = len(dynamic_retrieval_prompts)
                for prompt in dynamic_retrieval_prompts:
                    progress += 1
                    print(f"\rPrompt {progress}/{lenn}", end="")
                    prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=prompt)["embedding"]
                    chunks_most_similar_embeddings_dynamic = utils.find_most_similar(prompt_embedding, embeddings)[:max_chunks_count // 2]
                    chunks_most_similar_dynamic = [chunks[embedding[1]] for embedding in chunks_most_similar_embeddings_dynamic]
                    dynamic_chunks_most_similar.append(chunks_most_similar_dynamic)
                print(end="\n")

                chunks_most_similar = chunks_most_similar_static + [chunk for sublist in dynamic_chunks_most_similar for chunk in sublist]
                tokens_in_chunks = sum(utils.count_tokens(chunk) for chunk in chunks_most_similar)
                del chunks_most_similar_embeddings_static, chunks_most_similar_embeddings_dynamic  # free memory

                print(f"Tokens in average chunk group: {tokens_in_chunks / len(chunks_most_similar)}")
                final_prompts = []
                prompt_template = """
for question, chunks_most_similar_dynamic in zip(surv.questions, dynamic_chunks_most_similar):
    p = [
        systemMsg("\\n".join([
            f"You are an expert actor, specializing in impersonation of non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n#Context \\n##Chat conversations between the subject and their friends:\\n**From most to least related**\\n",
            "\\n\\nNEW CONVERSATION RELATED TO THE SURVEY OVERALL:\\n".join(chunks_most_similar_static),
            "\\n\\nNEW CONVERSATION RELATED TO THE PARTICULAR QUESTION:\\n".join(chunks_most_similar_dynamic)
        ])),      
        assistantMsg("Understood. I will answer from the point of view of the persona, based on what I could the deduct from the text provided."),
        userMsg("\\n".join([
            f"Persona is questioned about their {SURVEY} in an {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n",
            question,
            "\\nThe persona chooses:"
        ]))]
    final_prompts.append(p)
                """
            else: print("neither hybrid, nor dynamic")
            exec(prompt_template)
            prompt_info = utils.describe_prompts_and_print(final_prompts) # Vanity print
            SIM_ID = f"{SUBJECT}-{WHICH_SURVEY}-{RETRIAVAL_METHOD}-{chunk_size}-{str(overlap_size).zfill(2)}-{str(max_chunks_count).zfill(2)}_{MODEL}_V6"
            bu.quickJSON(final_prompts, f"data/5_monster_prep/{SIM_ID}_prompts.json")
            instructions = {
                "prompt_file": f"batch/prompts/{SIM_ID}_prompts.json",
                "survey_type": f"{SURVEY_TYPE[0]}",
                "isLocal": True,
                "LIMIT": None
            }
            settings = {
                "model": MODEL,
                "timeout": 300}
            AUTO_INFO = {
                "CHUNK_SIZE": chunk_size,
                "OVERLAP_SIZE": overlap_size,
                "CHUNKS_COUNT_IN_CTX": max_chunks_count,#chunks_count_in_ctx,
                "CTX_limit": max_tokens,
                "tokens_in_chunks": tokens_in_chunks,
                "model": EMBED_MODEL,
                "prompt method": PROMPT_METHOD,
                "retrieval method": RETRIAVAL_METHOD,
                "retrieval prompt": RETRIEVAL_PROMPT,
                "prompt_count": PROMPT_COUNT,
                "survey": WHICH_SURVEY,
                "subject": SUBJECT,
                "prompt_template": prompt_template,
                **prompt_info,
                **utils.describe_prompts([])
            }
            bu.quickJSON({"instructions": instructions, "settings": settings, "info": AUTO_INFO}, f"data/5_monster_prep/batch-schema/{SIM_ID}_schema.json")

In [ ]:
##### Display Info
total_messages = sum(len(chat) for chat in et.selectedChats.values())
chunks_count = len(chunks)
avg_chunk_char_len = np.mean([len(chunk) for chunk in chunks])
print(
    f"Chunk count: {chunks_count}",
    f"Rough estimate of tokens per chunk: {round(avg_chunk_char_len / 4)} (4 characters per token)",
    f"Messagees in input count: {total_messages}",
    f"Messages in chunks count: {stat_total_msgs_in_chunks}",
    f"Chunk \ Input ratio: {round(stat_total_msgs_in_chunks / total_messages,2)} (OVERLAP_SIZE={OVERLAP_SIZE})",
    f"Chunk Python type: {type(chunks[0])}",
    sep="\n"
) 
bu.if_dir_not_exist_make("data/3_embeddings")
bu.quickJSON(AUTO_INFO, f"data/3_embeddings/{EMBEDDING_ID}_info.json")
bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"data/3_embeddings/{EMBEDDING_ID}_embeddings.json")

In [ ]:
print(f"Tokens in chunks: {tokens_in_chunks}")
print(f"Chunks:{len(chunks)}, embeds:{len(embeddings)}")

# bu.quickTXT("\n\n".join(chunks_most_similar), filename=f"data/4_chunks/{CHECKPOINT}-static_chunks.txt")
# bu.if_dir_not_exist_make("data/4_chunks")
# bu.quickJSON(AUTO_INFO, f"data/4_chunks/{CHECKPOINT}-static_info.json")
# bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"data/4_chunks/{CHECKPOINT}-static_embeddings.json")

In [ ]:
# Display results
# bu.if_dir_not_exist_make("data/4_chunks")
# bu.quickJSON(AUTO_INFO, f"data/4_chunks/{CHECKPOINT}-dynamic_info.json")
# bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"data/4_chunks/{CHECKPOINT}-dynamic_embeddings.json")
############################################ VANITY BELOW ########################################
tokens_in_chunks = 0
for chunks_most_similar in dynamic_chunks_most_similar:
    for chunk in chunks_most_similar:
        tokens_in_chunks += utils.count_tokens(chunk)

del chunks_most_similar_embeddings # free memory
print(f"Tokens in average chunk group: {tokens_in_chunks/len(dynamic_chunks_most_similar)}")
# bu.quickJSON(dynamic_chunks_most_similar, filename=f"data/4_chunks/{CHECKPOINT}-dynamic_chunks.json")

In [ ]:
########################################### Method A ############################################
pre_final_prompts = []
if PROMPT_METHOD == "IMPERSONATE":
    pre_prompt_template = """
    SYS_MSG = {
        "role": "system", 
        "content": f"You are an expert actor, specializing in impersonation of non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n\\n**The persona, which you will be tasked to mimick is named '{SUBJECT}'.** \\n#Context \\n##Chat conversations between the subject and their friends:\\n**From most to least related**\\n"
    }
    ASSIST_MSG = {
        "role": "assistant",
        "content": "Understood. I will answer from the point of view of the persona, {subject}, based on what I could the deduct from the text provided."
    }
    USER_MSG = {
        "role": "user",
        "content": f"Persona is questioned about their {SURVEY} in an {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction."
    }
    pre_final_prompts.append(SYS_MSG, ASSIST_MSG, USER_MSG)
    """
    exec(pre_prompt_template)

########################################### Method B ###########################################
elif PROMPT_METHOD == "ARE":
    pre_prompt_template = """
    SYS_MSG = {
        "role": "system", 
        "content": f"**{TINY_MODULE}**. You have shared your thoughts, feelings, and experiences through text messages with friedns. Answer the following questions honestly and naturally, as you would in everyday conversations. \\n\\n#Context \\n##Conversations between persona and friends:"
    }
    ASSIST_MSG = {
        "role": "assistant",
        "content": f"Understood. I am {SUBJECT}, and I will answer the survey to the best of my ability."
    }
    USER_MSG = {   
        "role": "user",
        "content": f"The survey is about your {SURVEY}. You must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Your answer must only contain the chosen option, without any elaboration, nor introduction.\\n**From most to least related**\\n"
    }
    """
exec(pre_prompt_template)

print(f"{SYS_MSG['content']}")

In [ ]:
#BASE
base_final_prompts = []
prompt_template = """
for question in surv.questions:
    p = [
        systemMsg(
            "You are participating in a survey. You will be presented with a series of questions about your {SURVEY}.",
            f"You must choose answer to the question below with one of the five options: {', '.join(surv.POSSIBLE_ANSWERS)}. The answer must only contain the chosen option. "
        ),
        assistantMsg('Understood. I will answer the question below with one of the given options.'),
        userMsg(
            question,
            "Your choice: "
        )]
    base_final_prompts.append(p)
"""
exec(prompt_template)
# prompt_info = utils.describe_prompts_and_print(base_final_prompts) # Vanity print
# bu.quickJSON(base_final_prompts, f"data/5_prep/{WHICH_SURVEY}_base_prompts.json")

utils.count_tokens(prompt_template)
#Static
# static_final_prompts = []
# prompt_template = """
# for question in surv.questions:
#     p = [
#         systemMsg("\\n".join([
#             f"You are an expert actor, specializing in impersonation of non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n\\n**The persona, which you will be tasked to mimick is named '{SUBJECT}'.** \\n#Context \\n##Chat conversations between the subject and their friends:\\n**From most to least related**\\n",
#             "\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
#         ])),  
#         assistantMsg("Understood. I will answer from the point of view of the persona, {subject}, based on what I could the deduct from the text provided."),
#         userMsg("\\n".join([
#             f"Persona is questioned about their {SURVEY} in an {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n\\n",
#             question,
#             "\\nThe persona chooses:"
#         ]))]
#     static_final_prompts.append(p)
# """
# exec(prompt_template)
# prompt_info = utils.describe_prompts_and_print(static_final_prompts) # Vanity print
# bu.quickJSON(final_prompts, f"data/5_prep/{PREP_CHECKPOINT}_prompts.json")
# # print(f"{len(static_final_prompts)}")#,{final_prompts[:1]}")

# #Dynamic
# dynamic_final_prompts = []
# prompt_template = """
# for question, chunks_most_similar in zip(surv.questions, dynamic_chunks_most_similar):
#     p = [
#         systemMsg("\\n".join([
#             f"{SYS_MSG['content']}",
#             "\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
#         ])),      
#         assistantMsg(ASSIST_MSG['content']),
#         userMsg("\\n".join([
#             f"{USER_MSG['content']}\\n\\n**Your question is:**\\n\\n",
#             question,
#             "\\nThe persona chooses:"
#         ]))]
#     dynamic_final_prompts.append(p)
# """    
# exec(prompt_template)
# prompt_info = utils.describe_prompts_and_print(dynamic_final_prompts)
# bu.quickJSON(dynamic_final_prompts, f"data/5_prep/{PREP_CHECKPOINT}_prompts.json")
# # print(f"{len(dynamic_final_prompts)}")#,{final_prompts[:1]}")

In [ ]:
print(f"{SURVEY_TYPE[0]}")

In [ ]:
import os
import json

# Set the directory where the JSON files are located
directory = 'batch/done/monster_7b'  # Replace with the actual directory path if needed

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file has a .json extension
    if filename.endswith('.json'):
        # Open the JSON file
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as f:
            data = json.load(f)

        # Modify the "model" value
        if "settings" in data and "model" in data["settings"]:
            data["settings"]["model"] = "llama3"

        # Write the modified data back to the file
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=2)

In [ ]:
surv = survey.PersonalitySurvey()

In [ ]:
surv.df

